In [1]:
# functions.py

%run classes.ipynb

import os.path
from opencage.geocoder import OpenCageGeocode
import folium
from folium import Tooltip
import pickle 
from typing import List
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, Output
from IPython.display import clear_output, display
from datetime import datetime
import pytz
from typing import List
from copy import deepcopy

api_key = "Your API code here"

def get_lat_long_opencage(address):
    geocoder = OpenCageGeocode(api_key)
    try:
        geocode_result = geocoder.geocode(address)
        if geocode_result:
            location = geocode_result[0]['geometry']
            return location['lat'], location['lng']
        else:
            return 0, 0
    except Exception as e:
        print(f"Error geocoding address '{address}': {e}")
        return 0, 0

def create_map(full_client_list):
    map_nyc = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

    for fullClient in full_client_list:
        tooltip = folium.Tooltip(fullClient.name)
        popup_content = f"<strong>Name:</strong> {fullClient.name}<br>" \
                        f"<strong>Time:</strong> {fullClient.time}<br>" \
                        f"<strong>Length:</strong> {fullClient.length} minutes<br>" \
                        f"<strong>Address:</strong> {fullClient.address}"
        popup = folium.Popup(popup_content, max_width=250)
        
        # Check if the client is known
        if fullClient.known == "True":
            folium.Marker(location=[fullClient.lat, fullClient.long], popup=popup, tooltip=tooltip, icon=folium.Icon(color='red')).add_to(map_nyc)
        else:
            folium.Marker(location=[fullClient.lat, fullClient.long], popup=popup, tooltip=tooltip).add_to(map_nyc)
    
    # Render map in HTML iframe with defined width and height
    return folium.Figure(width=400, height=250).add_child(map_nyc)

def duplicate_remove():
    names_seen = {}
    to_remove = []

    for obj in mess_list:
        if obj.name not in names_seen:
            names_seen[obj.name] = obj
        else:
            names_seen[obj.name].time += ', ' + obj.time  # Use += instead of append
            to_remove.append(obj)

    for obj in to_remove:
        mess_list.remove(obj)

    return mess_list

def add_client():
    global full_client_list
    name = input("Enter name: ")

    existing_client = None
    for client in full_client_list:
        if client.name == name:
            print(client)
            existing_client = client

    if existing_client:
        continue_prompt = input("A client with this name already exists. Do you want to add time data (Y/N)? ")
        if continue_prompt.lower() == 'y':
            time_input = input("Enter times (in format: 'weekday month/day time') separated by commas: ")
            time = [t.strip() for t in time_input.split(",")] # convert string to list of times
            existing_client.time.extend(time) # Extend existing time data
        return
    
    time_input = input("Enter times (in format: 'weekday month/day time') separated by commas: ")
    time = [t.strip() for t in time_input.split(",")] # convert string to list of times
    known = input("Enter True if known, False if unknown: ")
    address = input("Enter address: ")
    length = input("Enter length in minutes: ")
    lat, long = get_lat_long_opencage(address)

    new_client = FullClient(name, time, known, address, length, lat, long)
    full_client_list.append(new_client)
    
    with open('client_data.pickle', 'wb') as f:
        pickle.dump(full_client_list, f)



def search():
    global full_client_list
    name = input("Enter the name of the client to search: ")

    # Search for the client in the list
    client_found = False
    for client in full_client_list:
        if client.name == name:
            print(client)
            client_found = True

    if not client_found:
        print("Client not found.")

def remove():
    global full_client_list
    name = input("Enter the name of the client to remove: ")

    # Search for the client in the list
    client_to_remove = None
    for client in full_client_list:
        if client.name == name:
            client_to_remove = client
            break

    # If client found, ask for confirmation to remove
    if client_to_remove:
        confirmation = input(f"Are you sure you want to remove {client_to_remove.name}? (Y/N): ")
        if confirmation.lower() == "y":
            full_client_list.remove(client_to_remove)
            print(f"{client_to_remove.name} has been removed.")
            # Save the updated list back to file
            with open('client_data.pickle', 'wb') as f:
                pickle.dump(full_client_list, f)
        else:
            print("No changes were made.")
    else:
        print("Client not found.")

def remove_append():
    global full_client_list
    seen_names = {}
    new_client_list = []

    for client in full_client_list:
        if client.name in seen_names:
            seen_names[client.name].time.extend(client.time)
        else:
            seen_names[client.name] = client
            new_client_list.append(client)

    full_client_list = new_client_list

def load_clients_pickle(file_path):
    try:
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        print("No file found at the provided path.")
        return []

def update_clients(old_clients):
    new_clients = []
    for client in old_clients:
        new_client = FullClient(client.name, client.time, client.known, client.address, client.length, client.lat, client.long)
        new_clients.append(new_client)
    return new_clients

def load_old_clients(filename):
    with open(filename, 'rb') as f:
        old_clients = pickle.load(f)
        return old_clients

def filter_and_display(clients: List[FullClient], day: str):
    filtered_clients = [client for client in clients if any(day.lower() in t.lower() for t in client.time)]
    map_nyc = create_map(filtered_clients)
    display(map_nyc)
    
def filter_and_save(clients: List[FullClient], day: str):
    filtered_clients = [client for client in clients if any(day.lower() in t.lower() for t in client.time)]
    filter_map = create_map(filtered_clients)
    return filter_map

def filter_by_time(clients: List[TimeClient], target_hour: int) -> List[TimeClient]:
    print(f"Target hour: {target_hour}")
    filtered_clients = [client for client in clients if int(client.hour) == target_hour]
    print(f"Number of matching clients: {len(filtered_clients)}")
    return filtered_clients

    
def weekly(change):
    day = days_of_week[change['new']]
    with out:
        clear_output(wait=True)
        if day == 'Sunday':
            print(day)
            display(sunday_map)
        if day == 'Monday':
            print(day)
            display(monday_map)
        if day == 'Tuesday':
            print(day)
            display(tuesday_map)
        if day == 'Wednesday':
            print(day)
            display(wednesday_map)
        if day == 'Thursday':
            print(day)
            display(thursday_map)
        if day == 'Friday':
            print(day)
            display(friday_map)
        if day == 'Saturday':
            print(day)
            display(saturday_map)

def hourly(change):
    with out2:
        clear_output(wait=True)
        target_hour = change.new
        #print("Target hour:", target_hour)
        #for client in time_client_list[:10]:  # Print the hour for the first 10 clients for debugging
            #print(f"Client hour: {client.hour}")
        matching_clients = [client for client in time_client_list if client.hour == target_hour]
        #print("Number of matching clients:", len(matching_clients))
        hour_map = filter_by_time(time_client_list, target_hour)
        time_map = create_time_map(hour_map)
        display(time_map)


def modify_time(clients: List[FullClient]) -> List[TimeClient]:
    time_client_list = []
    for client in deepcopy(clients):
        for t in client.time:
            parts = t.split()
            if len(parts) == 3:  # Only continue if there are exactly 3 parts
                weekday, date, time_str = parts
                hour, _ = time_str.split(':')
                hour = int(hour)
                
                # If the time is PM and it's not 12:00 PM, convert to 24-hour format
                if time_str.endswith('pm') and hour != 12:
                    hour += 12
                # If the time is 12:00 AM, convert to 0 in 24-hour format
                elif time_str.endswith('am') and hour == 12:
                    hour = 0
                
                # Ensure the latitude and longitude are stored as floats
                lat = float(client.lat)
                long = float(client.long)
                
                time_client_list.append(TimeClient(client.name, client.time, client.known, client.address, client.length, lat, long, weekday, date, str(hour)))
            else:
                print(f"Unexpected time format in client {client.name}: {t}")
    return time_client_list

def create_time_map(time_client_list):
    map_nyc = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

    for timeClient in time_client_list:
        tooltip = folium.Tooltip(timeClient.name)
        popup_content = f"<strong>Name:</strong> {timeClient.name}<br>" \
                        f"<strong>Time:</strong> {timeClient.time}<br>" \
                        f"<strong>Length:</strong> {timeClient.length} minutes<br>" \
                        f"<strong>Address:</strong> {timeClient.address}<br>" \
                        f"<strong>Weekday:</strong> {timeClient.weekday}<br>" \
                        f"<strong>Date:</strong> {timeClient.date}<br>" \
                        f"<strong>Hour:</strong> {timeClient.hour}"
        popup = folium.Popup(popup_content, max_width=250)
        
        # Check if the client is known
        if timeClient.known == "True":
            folium.Marker(location=[timeClient.lat, timeClient.long], popup=popup, tooltip=tooltip, icon=folium.Icon(color='red')).add_to(map_nyc)
        else:
            folium.Marker(location=[timeClient.lat, timeClient.long], popup=popup, tooltip=tooltip).add_to(map_nyc)
    
    # Render map in HTML iframe with defined width and height
    return folium.Figure(width=400, height=250).add_child(map_nyc)

def filter_by_day_and_time(change):
    target_day = days_of_week[day_slider.value]
    target_hour = time_slider_2.value
    with out3:
        clear_output(wait=True)
        matching_clients = [client for client in time_client_list if client.weekday.lower() == target_day.lower() and int(client.hour) == target_hour]
        day_time_map = create_time_map(matching_clients)
        display(day_time_map)


def day_time(change):
    target_day = days_of_week[slider_day_time.value[0]]
    target_hour = slider_day_time.value[1]
    with right_box:
        clear_output(wait=True)
        matching_clients = filter_by_day_and_time(time_client_list, target_day, target_hour)
        day_time_map = create_time_map(matching_clients)
        display(day_time_map)

        
    